In [ ]:
import pandas as pd
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


train_df = pd.read_csv("../data/interim/train_split.csv")
val_df   = pd.read_csv("../data/interim/validation_split.csv")

target_col = "Cover_Type"

# Separate features & target
X_train_scaled = train_df.drop(target_col, axis=1)
y_train = train_df[target_col]

X_val_scaled = val_df.drop(target_col, axis=1)
y_val   = val_df[target_col]


# ---- Logistic Regression (Multiclass) ----
log_reg = LogisticRegression(
    solver="lbfgs",
    max_iter=1000,
    random_state=42
)

log_reg.fit(X_train_scaled, y_train)
val_pred_log = log_reg.predict(X_val_scaled)
val_acc_log  = accuracy_score(y_val, val_pred_log)
print(f"Logistic Regression Validation Accuracy: {val_acc_log:.4f}")

joblib.dump(log_reg, "../models/logreg_multiclass.joblib")
print("Saved Logistic Regression model as logreg_multiclass.joblib")

#0.7238

### SVM

In [ ]:

svm_rbf = SVC(
    kernel="rbf",
    C=1.0,
    gamma="scale",
    decision_function_shape="ovr",
    random_state=42
)

svm_rbf.fit(X_train_scaled, y_train)
val_pred_svm = svm_rbf.predict(X_val_scaled)
val_acc_svm  = accuracy_score(y_val, val_pred_svm)
print(f"SVM (RBF) Validation Accuracy: {val_acc_svm:.4f}")

joblib.dump(svm_rbf, "../models/svm_rbf_multiclass.joblib")
#0.8306


### NN

In [2]:


nn_model = MLPClassifier(
    hidden_layer_sizes=(100,),
    activation="relu",
    solver="adam",
    max_iter=500,
    random_state=42
)

nn_model.fit(X_train_scaled, y_train)
val_pred_nn = nn_model.predict(X_val_scaled)
val_acc_nn  = accuracy_score(y_val, val_pred_nn)
print(f"Neural Network Validation Accuracy: {val_acc_nn:.4f}")

joblib.dump(nn_model, "../models/mlp_multiclass.joblib")
print("Saved Neural Network model as mlp_multiclass.joblib")
#0.8763

Neural Network Validation Accuracy: 0.8763
Saved Neural Network model as mlp_multiclass.joblib


In [3]:
import optuna
import joblib
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# --------------------------------------------------------------------
# Objective: define what Optuna should optimize
# --------------------------------------------------------------------
def objective(trial):
    # 1) Hyperparameter search space
    hidden_layer_sizes = trial.suggest_categorical(
        "hidden_layer_sizes",
        [(50,), (100,), (100, 50), (150, 100, 50), (200, 100)]
    )
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    solver = trial.suggest_categorical("solver", ["adam", "sgd"])
    learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-4, 1e-2)
    alpha = trial.suggest_loguniform("alpha", 1e-5, 1e-2)

    # 2) Create model with trial config
    model = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        learning_rate_init=learning_rate_init,
        alpha=alpha,
        max_iter=400,               # max epochs
        random_state=42,
        early_stopping=True,         # stop early if no improvement
        n_iter_no_change=20,
        verbose=False
    )

    # 3) Train
    model.fit(X_train_scaled, y_train)

    # 4) Validate
    val_pred = model.predict(X_val_scaled)
    val_acc = accuracy_score(y_val, val_pred)

    return val_acc

# --------------------------------------------------------------------
# Run Optuna Study
# --------------------------------------------------------------------
study = optuna.create_study(direction="maximize")
study.optimize(
    objective,
    n_trials=30,       # adjust based on time/budget
    timeout=3600       # seconds (optional: end after ~1 hour)
)

print("Best Validation Accuracy:", study.best_value)
print("Best Hyperparameters:", study.best_params)

# --------------------------------------------------------------------
# Train the Best Model on Full Train + Validation Combined (Optional)
# --------------------------------------------------------------------
best_params = study.best_params

best_nn_model = MLPClassifier(
    hidden_layer_sizes=best_params["hidden_layer_sizes"],
    activation=best_params["activation"],
    solver=best_params["solver"],
    learning_rate_init=best_params["learning_rate_init"],
    alpha=best_params["alpha"],
    max_iter=500,
    random_state=42,
    early_stopping=True,
    n_iter_no_change=20
)

# Fit final tuned model
best_nn_model.fit(np.vstack((X_train_scaled, X_val_scaled)),
                  np.hstack((y_train, y_val)))

# Save final model
joblib.dump(best_nn_model, "../models/mpl_tuned_optuna.joblib")

print("Saved tuned Neural Network model as mpl_tuned_optuna.joblib")


[I 2025-12-12 17:11:06,536] A new study created in memory with name: no-name-4338f6aa-cc5f-4017-a97c-3e1702b4ab35
/home/anuja/ml/smoker_status_prediction/venv/lib/python3.12/site-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
/home/anuja/ml/smoker_status_prediction/venv/lib/python3.12/site-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
/home/anuja/ml/smoker_status_prediction/venv/lib/python3.12/site-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 50) which is of type tuple.
  warnings.w

Best Validation Accuracy: 0.9263585459886176
Best Hyperparameters: {'hidden_layer_sizes': (200, 100), 'activation': 'relu', 'solver': 'adam', 'learning_rate_init': 0.0025986212005703682, 'alpha': 0.004727100327725838}
Saved tuned Neural Network model as mpl_tuned_optuna.joblib


In [ ]:

svm_rbf = SVC(
    kernel="rbf",
    C=1.0,
    gamma="scale",
    decision_function_shape="ovr",
    random_state=42
)

svm_rbf.fit(X_train_scaled, y_train)
val_pred_svm = svm_rbf.predict(X_val_scaled)
val_acc_svm  = accuracy_score(y_val, val_pred_svm)
print(f"SVM (RBF) Validation Accuracy: {val_acc_svm:.4f}")

joblib.dump(svm_rbf, "../models/svm_rbf_multiclass.joblib")
#0.8306


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Instantiate the SVM model
svm = SVC()

# Define the parameter grid
parameters = {'C': [0.1, 1, 10], 'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1]}

# Instantiate GridSearchCV
searcher = GridSearchCV(svm, parameters, cv=5)

# Run the search on the training data
searcher.fit(X_train_scaled, y_train)

# Print the best parameters found
print("Best CV params:", searcher.best_params_)

# Print the cross-validation accuracy
print("Best CV accuracy:", searcher.best_score_)

# Evaluate accuracy on the test set
test_accuracy = searcher.score(X_val_scaled, y_val)

print("Test accuracy of best grid search hypers:", test_accuracy)

